In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import pymysql
pymysql.install_as_MySQLdb()

### Store CSV into DataFrame

In [2]:
#file path
health = "Resources/health_nutrition.csv"
poverty = "Resources/poverty.csv"
world_development = "Resources/world_development_ind.csv"

#read file into pandas dataframe
health_df = pd.read_csv(health)
poverty_df = pd.read_csv(poverty)
world_development_df = pd.read_csv(world_development)

### Transform World Development Data Set

- organize columns and rows in the same way as the olympic data set

In [3]:
#size of rows for for loop
wd_size = world_development_df.shape[0]-5+55

#number of variables (iteration for for loop)
var = world_development_df['Series Name'].unique().shape[0]-1

In [4]:
#original formatting (we need column "series name" to be the column headers and the years to be the rows)
world_development_df.head()

,Country Name,Country Code,Series Name,Series Code,1960 [YR1960],1961 [YR1961],1962 [YR1962],1963 [YR1963],1964 [YR1964],1965 [YR1965],...,2009 [YR2009],2010 [YR2010],2011 [YR2011],2012 [YR2012],2013 [YR2013],2014 [YR2014],2015 [YR2015],2016 [YR2016],2017 [YR2017],2018 [YR2018]
0,Argentina,ARG,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,61.378,61.238,61.098,60.9582,60.8184,60.6786,...,63.8308,63.8792,63.9276,63.976,63.7388,63.5016,63.2644,63.0272,..,..
1,Argentina,ARG,"Agriculture, forestry, and fishing, value adde...",NV.AGR.TOTL.ZS,..,..,..,..,..,12.9041629886915,...,5.27362346890139,7.13216745078964,6.99873377022519,5.7817442068501,6.05291843670451,6.71270351428559,5.1566859021408,6.35390413957503,5.60873087646354,..
2,Argentina,ARG,"Annual freshwater withdrawals, total (% of int...",ER.H2O.FWTL.ZS,..,..,..,..,..,..,...,..,..,..,12.9075342465753,..,12.9075342465753,..,..,..,..
3,Argentina,ARG,Births attended by skilled health staff (% of ...,SH.STA.BRTC.ZS,..,..,..,..,..,..,...,97.9,95,97.1,98.2,97,99.6,99.6,..,..,..
4,Argentina,ARG,CO2 emissions (metric tons per capita),EN.ATM.CO2E.PC,2.36747303164667,2.44261589837139,2.52239210450438,2.31635633230104,2.53837930331286,2.64171434418192,...,4.41088989357125,4.55849961171786,4.60029069868628,4.56938367601348,4.46290375453177,4.74679745467325,..,..,..,..


In [5]:
#tranpose data set
world_development_T = world_development_df.T

In [6]:
#get first set of columns
world_development_new = world_development_T.iloc[:,0:55]
#set column names to series name
world_development_new.columns = world_development_new.iloc[2]
#store country name
country_name = world_development_new.iloc[0][1]
#store country code
country_2 = world_development_new.iloc[1][1]
#drop all string rows bc we stored them
world_development_new = world_development_new.drop(world_development_new.index[0:4])
#create column for country code
world_development_new.insert(loc=0, column='country_code', value=country_2)
#create column for country
world_development_new.insert(loc=0, column='country_name', value=country_name)

In [7]:
for i in range(110, wd_size, var):
    #repeat same process above for all countries
    joey = world_development_T.iloc[:,i-55:i]
    joey.columns = joey.iloc[2]
    country_name = joey.iloc[0][1]
    country_2 = joey.iloc[1][1]
    joey = joey.drop(joey.index[0:4])
    joey.insert(loc=0, column='country_code', value=country_2)
    joey.insert(loc=0, column='country_name', value=country_name)
    #append as rows to the bottom and iterate through each country
    world_development_new = world_development_new.append(joey, sort=False)

In [8]:
#new format of data set
world_development_new.head()

Series Name,country_name,country_code,"Adolescent fertility rate (births per 1,000 women ages 15-19)","Agriculture, forestry, and fishing, value added (% of GDP)","Annual freshwater withdrawals, total (% of internal resources)",Births attended by skilled health staff (% of total),CO2 emissions (metric tons per capita),"Contraceptive prevalence, any methods (% of women ages 15-49)",Domestic credit provided by financial sector (% of GDP),Electric power consumption (kWh per capita),...,"School enrollment, primary (% gross)","School enrollment, primary and secondary (gross), gender parity index (GPI)","School enrollment, secondary (% gross)",Statistical Capacity score (Overall average),Surface area (sq. km),Tax revenue (% of GDP),Terrestrial and marine protected areas (% of total territorial area),Time required to start a business (days),"Total debt service (% of exports of goods, services and primary income)",Urban population growth (annual %)
1960 [YR1960],Argentina,ARG,61.378,..,..,..,2.36747303164667,..,21.6715343239319,..,...,..,..,..,..,..,..,..,..,..,2.68105176743018
1961 [YR1961],Argentina,ARG,61.238,..,..,..,2.44261589837139,..,18.7641792149707,..,...,..,..,..,..,2780400,..,..,..,..,2.42676637719918
1962 [YR1962],Argentina,ARG,61.098,..,..,..,2.52239210450438,..,17.2659768639598,..,...,..,..,..,..,2780400,..,..,..,..,2.32264606750133
1963 [YR1963],Argentina,ARG,60.9582,..,..,..,2.31635633230104,..,17.9507787335224,..,...,..,..,..,..,2780400,..,..,..,..,2.27983406180105
1964 [YR1964],Argentina,ARG,60.8184,..,..,..,2.53837930331286,..,17.442243973229,..,...,..,..,..,..,2780400,..,..,..,..,2.23211910300166


In [9]:
#replace string value with NaN
world_development_new = world_development_new.replace('..', np.nan)

In [10]:
#get columns we want
#list of columns
columns = list(world_development_new)
#pd.DataFrame(columns)
wd_data = world_development_new[[columns[0],columns[1], columns[3], columns[9], columns[10], columns[11],     
                                columns[13], columns[15], columns[16], columns[17], columns[18], columns[19],    
                                columns[20], columns[21], columns[22], columns[23], columns[24], columns[25],
                                columns[26], columns[27], columns[28], columns[29], columns[30], columns[31], 
                                columns[33], columns[34], columns[35], columns[38], columns[39], columns[40],
                                columns[41], columns[42], columns[44], columns[46], columns[47], columns[48],
                                columns[49], columns[51], columns[52], columns[55], columns[56]]]

In [11]:
#figure out which columns don't have enough data (format: percentage that is NaN)
#threshold is 60% NaN Values
bad_col = wd_data.isna().sum()/wd_data.shape[0]*100 > 60

In [12]:
#store the column names as a list
bad_col = bad_col[bad_col==True].index

In [13]:
#drop these columns
wd_data = wd_data.drop(bad_col, axis=1)

In [14]:
#reset index
wd_data = wd_data.reset_index(drop=False)

In [15]:
#list of new column names
col_names = ['year',
 'country_name',
 'country_code',
 'Agriculture_value_added_(%_of_GDP)',
 'Electric_power_consumption(kWh_per_capita)',
 'Energy_use(kg_of_oil_equivalent_per_capita)',
 'Exports_of_goods_and_services(%_of_GDP)',
 'Fertility_rate_total_births_per_woman',
 'Forest_area_(sq_km)',
 'GDP_(current_US$)',
 'GDP_growth_(annual_%)',
 'GNI_per_capita_Atlas_method_(current_US$)',
 'GNI_per_capita_PPP_(current_international_$)',
 'GNI_Atlas_method_(current_US$)',
 'GNI_PPP_(current_international_$)',
 'Gross_capital_formation_(%_of_GDP)',
 'Immunization_measles_(%_ages_12_23_months)',
 'Import_of_goods(%_of_GDP)',
 'Industry_value added_(%_of_GDP)',
 'Inflation_(annual_%)',
 'Life_expectancy_at_birth_total_years',
 'Merchandise_trade_(%_of_GDP)',
 'Military_expenditure_(%_of_GDP)',
 'Mortality_rate_under_5_(per_1,000_live births)',
 'Population_density_(people_per_sq_km_of_land_area)',
 'Population_growth_(annual_%)',
 'Population_total',
 'School_enrollment_primary_(%_gross)',
 'School_enrollment_primary_and_secondary_(GPI)',
 'School_enrollment_secondary_(%_gross)',
 'Surface_area_(sq_km)',
 'Urban_population_growth_(annual_%)']

#rename columns (too long for SQL)
wd_data.columns = col_names

In [16]:
wd_data['year'] = wd_data['year'].astype(str).str[:-9].astype(object)

In [17]:
#output final pandas DataFrame
wd_data.head()

,year,country_name,country_code,Agriculture_value_added_(%_of_GDP),Electric_power_consumption(kWh_per_capita),Energy_use(kg_of_oil_equivalent_per_capita),Exports_of_goods_and_services(%_of_GDP),Fertility_rate_total_births_per_woman,Forest_area_(sq_km),GDP_(current_US$),...,Military_expenditure_(%_of_GDP),"Mortality_rate_under_5_(per_1,000_live births)",Population_density_(people_per_sq_km_of_land_area),Population_growth_(annual_%),Population_total,School_enrollment_primary_(%_gross),School_enrollment_primary_and_secondary_(GPI),School_enrollment_secondary_(%_gross),Surface_area_(sq_km),Urban_population_growth_(annual_%)
0,1960,Argentina,ARG,NaN,NaN,NaN,7.60404910825644,3.109,NaN,NaN,...,1.55,NaN,NaN,1.63087449288958,20619075,NaN,NaN,NaN,NaN,2.68105176743018
1,1961,Argentina,ARG,NaN,NaN,NaN,5.9949469947787,3.1,NaN,NaN,...,2.14,NaN,7.65635749756092,1.60688908688843,20953077,NaN,NaN,NaN,2780400,2.42676637719918
2,1962,Argentina,ARG,NaN,NaN,NaN,4.69184275899831,3.089,NaN,24450604877.6081,...,2.66,NaN,7.77862381197724,1.58430864789372,21287682,NaN,NaN,NaN,2780400,2.32264606750133
3,1963,Argentina,ARG,NaN,NaN,NaN,7.89045420262462,3.078,NaN,18272123664.4715,...,1.68,NaN,7.90072679039277,1.55753194021187,21621840,NaN,NaN,NaN,2780400,2.27983406180105
4,1964,Argentina,ARG,NaN,NaN,NaN,5.56371560478508,3.068,NaN,25605249381.7597,...,1.15,NaN,8.02207374602165,1.5242206252397,21953929,NaN,NaN,NaN,2780400,2.23211910300166


### Create Data Base Connection

In [18]:
rds_connection_string = "root:Datascience33!?@localhost/"
engine = create_engine(f'mysql://{rds_connection_string}')

connection = engine.connect()

In [385]:
connection.execute('create schema world_development_db;')

In [19]:
db_connection_string = "root:Datascience33!?@localhost/world_development_db"
db_engine = create_engine(f'mysql://{db_connection_string}')

# Confirm tables
results =db_engine.execute('show databases')
for item in results:
    print(item)

('information_schema',)
('animals_db',)
('birds',)
('books_db',)
('customer_db',)
('favorite_db',)
('icecream_db',)
('joining_team',)
('miscellaneous_db',)
('mysql',)
('performance_schema',)
('sakila',)
('scud',)
('second_international_bank',)
('sys',)
('witch_craft',)
('world',)
('world_development_db',)


### Load DataFrames into DataBase

In [503]:
#use this function to delete table if needed
#db_engine.table_names()
#db_engine.execute('DROP TABLE IF EXISTS w_dev ')

In [20]:
wd_data.to_sql(name='world_dev_data', con=db_engine, if_exists='replace', index=True)

In [21]:
db_engine.table_names()

['olympic_data', 'world_dev_data']

In [22]:
#Confirm data has been added by querying the w_dev table
pd.read_sql_query('select * from world_dev_data', con=db_engine).head()

,index,year,country_name,country_code,Agriculture_value_added_(%_of_GDP),Electric_power_consumption(kWh_per_capita),Energy_use(kg_of_oil_equivalent_per_capita),Exports_of_goods_and_services(%_of_GDP),Fertility_rate_total_births_per_woman,Forest_area_(sq_km),...,Military_expenditure_(%_of_GDP),"Mortality_rate_under_5_(per_1,000_live births)",Population_density_(people_per_sq_km_of_land_area),Population_growth_(annual_%),Population_total,School_enrollment_primary_(%_gross),School_enrollment_primary_and_secondary_(GPI),School_enrollment_secondary_(%_gross),Surface_area_(sq_km),Urban_population_growth_(annual_%)
0,0,1960,Argentina,ARG,None,None,None,7.60404910825644,3.109,None,...,1.55,None,None,1.63087449288958,20619075,None,None,None,None,2.68105176743018
1,1,1961,Argentina,ARG,None,None,None,5.9949469947787,3.1,None,...,2.14,None,7.65635749756092,1.60688908688843,20953077,None,None,None,2780400,2.42676637719918
2,2,1962,Argentina,ARG,None,None,None,4.69184275899831,3.089,None,...,2.66,None,7.77862381197724,1.58430864789372,21287682,None,None,None,2780400,2.32264606750133
3,3,1963,Argentina,ARG,None,None,None,7.89045420262462,3.078,None,...,1.68,None,7.90072679039277,1.55753194021187,21621840,None,None,None,2780400,2.27983406180105
4,4,1964,Argentina,ARG,None,None,None,5.56371560478508,3.068,None,...,1.15,None,8.02207374602165,1.5242206252397,21953929,None,None,None,2780400,2.23211910300166


## Transform Olympic Data

In [71]:
olympic_file = "Resources/athlete_events.csv"
olympic_data_df = pd.read_csv(olympic_file)

In [72]:
olympic_data_df = olympic_data_df.rename(index=str, columns={"Team": "country_name2", "NOC": "country_code2", "Year":"year2"})

In [73]:
olympic_data_df.to_sql(name='olympic_data', con=db_engine, if_exists='replace', index=False)

In [74]:
#confirm dataframe has been added to database by querying
pd.read_sql_query('select * from olympic_data', con=db_engine).head()

,ID,Name,Sex,Age,Height,Weight,country_name2,country_code2,Games,year2,Season,City,Sport,Event,Medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,None
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,None
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,None
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,None


## Transform Health Personnel Data

In [51]:
un_file="Resources/SYB61_T10_Health Personnel.csv"
un_data_df=pd.read_csv(un_file,encoding="latin-1")
un_data_df=un_data_df.iloc[1:,0:5]

In [52]:
un_data_df.columns=['CountryID',"Country","Year","Health_Profession","Number_Professionals"]

In [53]:
un_data_df['Health_Profession'].unique()

array(['Health personnel: Dentists (per 1000 population)',
       'Health personnel: Physicians (number)',
       'Health personnel: Physicians (per 1000 population)',
       'Health personnel: Pharmacists (number)',
       'Health personnel: Pharmacists (per 1000 population)',
       'Health personnel: Nurses and midwives (number)',
       'Health personnel: Nurses and midwives personnel (per 1000 population)',
       'Health personnel: Dentists (number)'], dtype=object)

In [54]:
un_data_df.head()

,CountryID,Country,Year,Health_Profession,Number_Professionals
1,396,Johnston Island [former],2004,Health personnel: Dentists (per 1000 population),0.0
2,396,Johnston Island [former],2008,Health personnel: Dentists (per 1000 population),0.2
3,396,Johnston Island [former],2013,Health personnel: Dentists (per 1000 population),0.2
4,4,Afghanistan,2001,Health personnel: Physicians (number),"4,104"
5,4,Afghanistan,2001,Health personnel: Physicians (per 1000 populat...,0.2


In [55]:
un_data_df=un_data_df[un_data_df['Health_Profession'].str.contains('per 1000 population')==False]

In [56]:
un_data_df['Health_Profession'].unique()

array(['Health personnel: Physicians (number)',
       'Health personnel: Pharmacists (number)',
       'Health personnel: Nurses and midwives (number)',
       'Health personnel: Dentists (number)'], dtype=object)

In [57]:
prof_clean_df=pd.DataFrame({'Health_Profession':['Health personnel: Physicians (number)','Health personnel: Pharmacists (number)',
                                  'Health personnel: Nurses and midwives (number)', 'Health personnel: Dentists (number)'],
             'Profession':['Physicians',"Pharmacists","Nurses and midwives","Dentists"]})

In [58]:
un_data_df=pd.merge(un_data_df,prof_clean_df,on='Health_Profession',how='outer')

In [59]:
un_data_df=un_data_df.iloc[:,[0,1,2,5,4]]

In [60]:
un_data_df['country_year']=un_data_df['Country']+un_data_df["Year"]
un_data_df.head()
pivot_df=un_data_df.pivot(index='country_year',columns="Profession",values="Number_Professionals")
pivot_df['country_year']=pivot_df.index
clean_un_df=pd.merge(un_data_df.loc[:,['Country','Year','country_year']].drop_duplicates(),pivot_df,on='country_year',how='outer')
clean_un_df=clean_un_df.iloc[:,[0,1,3,4,5,6]]

C:\Users\ashcr\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3267: FutureWarning: 'country_year' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version
  exec(code_obj, self.user_global_ns, self.user_ns)


In [61]:
clean_un_df=clean_un_df.rename(columns={'Country':'country_name3','Year':'year3'})
clean_un_df.head()

,country_name3,year3,Dentists,Nurses and midwives,Pharmacists,Physicians
0,Afghanistan,2001,NaN,NaN,525,"4,104"
1,Afghanistan,2008,382,"13,780",829,"4,834"
2,Afghanistan,2009,NaN,"17,257",847,"6,037"
3,Afghanistan,2010,NaN,NaN,814,"6,901"
4,Afghanistan,2014,111,"11,399","1,711","9,954"


In [63]:
#delete weird country name
clean_un_df = clean_un_df[~clean_un_df.country_name3.str.contains("Côte")]

In [64]:
clean_un_df.head()

,country_name3,year3,Dentists,Nurses and midwives,Pharmacists,Physicians
0,Afghanistan,2001,NaN,NaN,525,"4,104"
1,Afghanistan,2008,382,"13,780",829,"4,834"
2,Afghanistan,2009,NaN,"17,257",847,"6,037"
3,Afghanistan,2010,NaN,NaN,814,"6,901"
4,Afghanistan,2014,111,"11,399","1,711","9,954"


In [65]:
clean_un_df.to_sql(name='health_data', con=db_engine, if_exists='replace', index=False)

In [66]:
#confirm dataframe has been added to database by querying
pd.read_sql_query('select * from health_data', con=db_engine).head()

,country_name3,year3,Dentists,Nurses and midwives,Pharmacists,Physicians
0,Afghanistan,2001,None,None,525,"4,104"
1,Afghanistan,2008,382,"13,780",829,"4,834"
2,Afghanistan,2009,None,"17,257",847,"6,037"
3,Afghanistan,2010,None,None,814,"6,901"
4,Afghanistan,2014,111,"11,399","1,711","9,954"


## SQL Join Data Sets

In [82]:
merged_data_df = pd.read_sql_query('SELECT * \
                    FROM olympic_data od \
                    LEFT JOIN world_dev_data wd ON wd.country_name = od.country_name2 \
                    AND wd.`year` = od.`year2` \
                    LEFT JOIN health_data hd ON hd.country_name3 = od.country_name2 \
                    AND hd.`year3`= od.`year2` ;', con=db_engine)

In [83]:
print(merged_data_df.shape)
merged_data_df.head()

(271116, 54)


,ID,Name,Sex,Age,Height,Weight,country_name2,country_code2,Games,year2,...,School_enrollment_primary_and_secondary_(GPI),School_enrollment_secondary_(%_gross),Surface_area_(sq_km),Urban_population_growth_(annual_%),country_name3,year3,Dentists,Nurses and midwives,Pharmacists,Physicians
0,148,"Kathryn ""Katie"" Abbott",F,21.0,164.0,63.0,Canada,CAN,2008 Summer,2008,...,0.98624,102.12339,9984670,1.30831924280004,Canada,2008,"41,798","343,699","29,010","65,440"
1,314,"Rukayatu ""Ruky"" Abdulai",F,22.0,180.0,59.0,Canada,CAN,2008 Summer,2008,...,0.98624,102.12339,9984670,1.30831924280004,Canada,2008,"41,798","343,699","29,010","65,440"
2,395,Jennifer Abel,F,16.0,160.0,62.0,Canada,CAN,2008 Summer,2008,...,0.98624,102.12339,9984670,1.30831924280004,Canada,2008,"41,798","343,699","29,010","65,440"
3,9,Antti Sami Aalto,M,26.0,186.0,96.0,Finland,FIN,2002 Winter,2002,...,1.0566,128.79507,338150,0.406146958535345,Finland,2002,"6,674","78,402","5,829","16,446"
4,16,Juhamatti Tapio Aaltonen,M,28.0,184.0,85.0,Finland,FIN,2014 Winter,2014,...,1.05652,146.42318,338420,0.472281589887817,Finland,2014,"3,925","82,368","5,988","17,511"
